In [1]:
import pandas as pd

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
file = 'Задание_для_соискателя.xlsx'

zlog = 'zlog'
params = 'params'

In [4]:
# читаем файлы

df_zlog = pd.read_excel(file, sheet_name=zlog, skiprows=5)
df_params = pd.read_excel(file, sheet_name=params, skiprows=1)

In [5]:
# смотрим

display(df_zlog.head())
display(df_params.head())

,zid,action,time,bank
0,1,z-created,2020-12-09 16:13:45,0
1,1,stto-ac,2020-12-09 16:13:45,0
2,1,edited,2020-12-09 16:15:46,0
3,1,stto-cs,2020-12-09 16:16:00,0
4,2,z-created,2020-12-09 16:55:04,0


,zid,sumbg
0,1,"618,799.50"
1,2,"15,334,644.60"
2,3,"6,415,667.37"
3,4,"9,654,890.52"
4,5,"1,587,607.42"


# 1

In [6]:
# соединяем
df = df_zlog.merge(df_params, on='zid', how='left')

In [7]:
df

,zid,action,time,bank,sumbg
0,1,z-created,2020-12-09 16:13:45,0,"618,799.50"
1,1,stto-ac,2020-12-09 16:13:45,0,"618,799.50"
2,1,edited,2020-12-09 16:15:46,0,"618,799.50"
3,1,stto-cs,2020-12-09 16:16:00,0,"618,799.50"
4,2,z-created,2020-12-09 16:55:04,0,"15,334,644.60"
...,...,...,...,...,...
18749,103,stto-cr,2021-03-04 13:33:25,bank5,"4,481,896.64"
18750,103,stto-ar,2021-03-04 13:33:32,bank5,"4,481,896.64"
18751,850,stto-pa,2021-03-11 03:51:35,bank5,"139,657.10"
18752,857,stto-pa,2021-03-21 01:11:06,bank5,"72,602.40"


In [8]:
# необходимые события

cols = [
    'stto-bgs', # дата выдачи
    'stto-bms'  # дата одобрения
]

# фильтруем данные по событиям
df_filtered = df[df['action'].isin(cols)].reset_index(drop=True)

# сортируем по времени, далее для каждого договора и события создаем нумерацию по порядку
df_filtered['row_num'] = df_filtered.sort_values(by='time').groupby(['zid', 'action'])['bank'].cumcount() + 1

In [9]:
# пример
df_filtered[df_filtered['zid'] == 377]

,zid,action,time,bank,sumbg,row_num
859,377,stto-bms,2020-12-16 12:00:43,bank5,"22,190,813.04",1
927,377,stto-bms,2020-12-17 10:00:14,bank5,"22,190,813.04",2
948,377,stto-bms,2020-12-17 12:05:33,bank5,"22,190,813.04",3
1063,377,stto-bgs,2020-12-18 14:02:47,bank5,"22,190,813.04",1
1654,377,stto-bgs,2021-02-08 11:12:27,bank5,"22,190,813.04",2


In [10]:
# выкидываем все события кроме первого
df_clean = df_filtered[df_filtered['row_num'] == 1].reset_index(drop=True)

# выкидываем нумерацию
df_clean = df_clean[['zid', 'action', 'time', 'bank', 'sumbg']]

In [11]:
df_clean

,zid,action,time,bank,sumbg
0,23,stto-bms,2020-12-10 11:19:46,bank1,"7,096,956.68"
1,38,stto-bms,2020-12-10 13:00:15,bank1,"783,540.06"
2,24,stto-bms,2020-12-10 13:49:56,bank1,"7,356,104.60"
3,34,stto-bms,2020-12-10 13:52:44,bank1,"3,374,827.74"
4,35,stto-bms,2020-12-10 14:22:36,bank1,"11,873,974.02"
...,...,...,...,...,...
993,828,stto-bgs,2021-01-15 15:59:59,bank5,"20,295,738.04"
994,853,stto-bgs,2021-01-21 17:47:32,bank5,"137,543.45"
995,243,stto-bgs,2021-01-25 15:49:03,bank5,"3,694,806.50"
996,847,stto-bgs,2021-01-28 17:44:57,bank5,"446,964.00"


In [12]:
# даты одобрения
df_odobr = df_clean[df_clean['action'] == 'stto-bms'][['zid', 'time', 'bank']].rename(columns={'time': 'data_odobr'})

# даты выдачи
df_vidacha = df_clean[df_clean['action'] == 'stto-bgs'][['zid', 'time', 'sumbg']].rename(columns={'time': 'data_vidachi'})

# соединяем
df_final = df_odobr.merge(df_vidacha, on='zid', how='outer')

In [13]:
df_final

,zid,data_odobr,bank,data_vidachi,sumbg
0,23,2020-12-10 11:19:46,bank1,NaT,NaN
1,38,2020-12-10 13:00:15,bank1,NaT,NaN
2,24,2020-12-10 13:49:56,bank1,NaT,NaN
3,34,2020-12-10 13:52:44,bank1,NaT,NaN
4,35,2020-12-10 14:22:36,bank1,NaT,NaN
...,...,...,...,...,...
973,838,2020-12-25 14:00:09,bank5,NaT,NaN
974,865,2020-12-25 17:55:22,bank5,NaT,NaN
975,879,2020-12-28 12:58:01,bank5,2021-01-11 12:35:01,"9,836,665.08"
976,442,2020-12-28 16:55:14,bank5,NaT,NaN


In [14]:
# считаем № корзины по условию

df_final['bucket'] = df_final['sumbg'].apply(
    lambda x: (x < 500000 and 1) or 
              (x <= 1000000 and 2) or 
              (x <= 5000000 and 3) or 
              (x > 5000000 and 4)
)

In [15]:
#  выкидываем лишние колонки и сортируем по порядку из текста задачи
df_final = df_final[['zid', 'bank', 'sumbg', 'data_odobr', 'data_vidachi', 'bucket']]

In [16]:
df_final

,zid,bank,sumbg,data_odobr,data_vidachi,bucket
0,23,bank1,NaN,2020-12-10 11:19:46,NaT,False
1,38,bank1,NaN,2020-12-10 13:00:15,NaT,False
2,24,bank1,NaN,2020-12-10 13:49:56,NaT,False
3,34,bank1,NaN,2020-12-10 13:52:44,NaT,False
4,35,bank1,NaN,2020-12-10 14:22:36,NaT,False
...,...,...,...,...,...,...
973,838,bank5,NaN,2020-12-25 14:00:09,NaT,False
974,865,bank5,NaN,2020-12-25 17:55:22,NaT,False
975,879,bank5,"9,836,665.08",2020-12-28 12:58:01,2021-01-11 12:35:01,4
976,442,bank5,NaN,2020-12-28 16:55:14,NaT,False


# 2

In [17]:
# в экселе это автоматом делается, я тут геммороюсь
# получаю месяц
df_final['month'] = df_final['data_vidachi'].dt.month


C:\Users\SERGEY~1.SMY\AppData\Local\Temp/ipykernel_15904/3821828284.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['month'] = df_final['data_vidachi'].dt.month


In [18]:
# выкидываю всё где нет выдачи
df_pivot = df_final[df_final['month'].notna()].reset_index(drop=True)

# по умолчанию получается дробное число, я делаю его целочисленным т.е. 1.0 -> 1
df_pivot['month'] = df_pivot['month'].astype(int)

In [19]:
df_pivot

,zid,bank,sumbg,data_odobr,data_vidachi,bucket,month
0,732,bank1,"11,421,027.90",2020-12-18 17:55:25,2020-12-25 15:19:28,4,12
1,954,bank1,"1,553,186.00",2020-12-23 10:50:35,2021-01-20 12:46:59,3,1
2,752,bank1,"5,880,093.00",2021-01-18 15:25:09,2021-01-19 16:20:15,4,1
3,967,bank1,"3,499,221.06",2021-02-01 18:37:25,2021-02-04 17:17:06,3,2
4,965,bank3,"1,338,250.56",2020-12-24 10:00:55,2021-01-14 15:13:10,3,1
5,146,bank5,"21,853,823.43",2020-12-11 13:09:39,2020-12-14 10:53:27,4,12
6,90,bank5,"756,768.96",2020-12-11 14:25:13,2021-02-05 18:25:54,2,2
7,140,bank5,"58,291,235.00",2020-12-11 15:20:31,2020-12-21 17:04:57,4,12
8,243,bank5,"3,694,806.50",2020-12-14 11:37:08,2021-01-25 15:49:03,3,1
9,377,bank5,"22,190,813.04",2020-12-16 12:00:43,2020-12-18 14:02:47,4,12


In [20]:
# строим сводную таблицу, по идее это что-то похожее на ответ

df_pivot.pivot_table(
    values='sumbg',
    index=['bank', 'bucket'],
    columns=['month']
).fillna(0)

month                   1            2             12
bank  bucket                                         
bank1 3       1,553,186.00 3,499,221.06          0.00
      4       5,880,093.00         0.00 11,421,027.90
bank3 3       1,338,250.56         0.00          0.00
bank5 1         292,253.72         0.00          0.00
      2               0.00   756,768.96          0.00
      3       3,694,806.50         0.00  3,447,235.86
      4      12,180,289.90         0.00 27,586,740.16

# 3

In [21]:
# просто скопирую таблицу на всякий случай
df_conversion = df_final.copy()

In [22]:
# если бан выдал кредит - ставлю 1, если просто одобрил - то 0
df_conversion['credit_vidan'] = df_conversion['sumbg'].apply(lambda x: 1 if x > 0 else 0)

In [23]:
df_conversion

,zid,bank,sumbg,data_odobr,data_vidachi,bucket,month,credit_vidan
0,23,bank1,NaN,2020-12-10 11:19:46,NaT,False,NaN,0
1,38,bank1,NaN,2020-12-10 13:00:15,NaT,False,NaN,0
2,24,bank1,NaN,2020-12-10 13:49:56,NaT,False,NaN,0
3,34,bank1,NaN,2020-12-10 13:52:44,NaT,False,NaN,0
4,35,bank1,NaN,2020-12-10 14:22:36,NaT,False,NaN,0
...,...,...,...,...,...,...,...,...
973,838,bank5,NaN,2020-12-25 14:00:09,NaT,False,NaN,0
974,865,bank5,NaN,2020-12-25 17:55:22,NaT,False,NaN,0
975,879,bank5,"9,836,665.08",2020-12-28 12:58:01,2021-01-11 12:35:01,4,1.00,1
976,442,bank5,NaN,2020-12-28 16:55:14,NaT,False,NaN,0


In [24]:
# тут  просто сгруппировал, а нужно чтоб данные по одному банку были в одной строке
df_conversion.groupby(['bank', 'credit_vidan'])['zid'].count().reset_index()

,bank,credit_vidan,zid
0,bank1,0,200
1,bank1,1,4
2,bank2,0,20
3,bank3,0,82
4,bank3,1,1
5,bank5,0,656
6,bank5,1,15


In [25]:
# поэтому делаем пивот (т.е. сводную таблицу)
# я в них сильно плох, так и не нашёл как убрать "credit_vidan"


df_credits = df_conversion.pivot_table(
    index='bank',
    columns='credit_vidan',
    values='zid',
    aggfunc='count'
).reset_index().rename(columns={0: 'ne_vidan', 1: 'vidan'})

# заполняю нулями пропуски
df_credits.fillna(0, inplace=True)

# считаю общее количество действий банка
df_credits['total'] = df_credits['vidan'] + df_credits['ne_vidan']

In [26]:
df_credits

credit_vidan,bank,ne_vidan,vidan,total
0,bank1,200.00,4.00,204.00
1,bank2,20.00,0.00,20.00
2,bank3,82.00,1.00,83.00
3,bank5,656.00,15.00,671.00


In [27]:
# считаю отношение выдачи к невыдаче
df_credits['procent_vidachi_k_odobr'] = df_credits['vidan'] / df_credits['ne_vidan'] * 100

# считаю выдачу к общим действиям банка
df_credits['procent_odobr'] = df_credits['vidan'] / df_credits['total'] * 100

# это что-то похожее на ответ
df_credits

credit_vidan,bank,ne_vidan,vidan,total,procent_vidachi_k_odobr,procent_odobr
0,bank1,200.00,4.00,204.00,2.00,1.96
1,bank2,20.00,0.00,20.00,0.00,0.00
2,bank3,82.00,1.00,83.00,1.22,1.20
3,bank5,656.00,15.00,671.00,2.29,2.24
